In [21]:
import pickle
with open('gnnexplainer_result/msr_4x_split_0_hop_1.pkl', 'rb') as fp:
     gnn_explainer_dict = pickle.load(fp)

In [2]:
import pandas as pd

In [3]:
# the test set with vul line information
df_test = pd.read_json('msr_test_set_with_line.json')

In [4]:
df_test

,node_features,node_features_sym,graph,original_tokens,symbolic_tokens,targets,token,token_list,line
0,"[[-1.0918805779, -0.0600933482, -1.3017879377,...","[[-0.0134278598, -0.44801456980000004, -0.0053...","[[0, 2, 25], [25, 3, 0], [0, 2, 2], [2, 3, 0],...","[[static, void, usb_net_handle_dataout, (, USB...","[[static, void, FUNC1, (, VAR1, *, VAR2, ,, VA...",[[0]],"[static, void, usb_net_handle_dataout, (, USBN...","static,void,usb_net_handle_dataout,(,USBNetSta...",None
1,"[[0.5474294862, 0.32906730910000004, -1.650767...","[[0.1319378658, -0.397455169, -0.0868625781, -...","[[0, 2, 2], [2, 3, 0], [0, 2, 4], [4, 3, 0], [...","[[static, int, do_flock, (, struct, file, *, f...","[[static, int, FUNC1, (, struct, VAR1, *, VAR1...",[[0]],"[static, int, do_flock, (, struct, file, *, fi...","static,int,do_flock,(,struct,file,*,file,,,int...",None
2,"[[-0.7882826924, -0.54334445, -1.1545437828, 0...","[[-0.3183312699, -0.24551808390000002, -0.0939...","[[0, 2, 8], [8, 3, 0], [0, 2, 9], [9, 3, 0]]","[[void, AddSystemStrings, (, content, ::, WebU...","[[void, FUNC1, (, VAR1, :, VAR2, *, VAR3, ), {...",[[0]],"[void, AddSystemStrings, (, content, ::, WebUI...","void,AddSystemStrings,(,content,::,WebUIDataSo...",None
3,"[[-0.4252009115, -1.3483376971, -0.8432620365,...","[[0.0869873634, -0.3488178274, -0.1156845348, ...","[[0, 2, 3], [3, 3, 0], [0, 2, 9], [9, 3, 0], [...","[[SMB2_sess_alloc_buffer, (, struct, SMB2_sess...","[[FUNC1, (, struct, VAR1, *, VAR2, )], [{], [i...",[[0]],"[SMB2_sess_alloc_buffer, (, struct, SMB2_sess_...","SMB2_sess_alloc_buffer,(,struct,SMB2_sess_data...",None
4,"[[1.1260667245, 0.1808472077, 0.03605219720000...","[[0.2014318407, -0.2869973059, 0.1425949335, -...",[],"[[AutofillPopupWarningView, (, AutofillPopupVi...","[[FUNC1, (, VAR1, *, VAR2, ,], [int, VAR1, )],...",[[0]],"[AutofillPopupWarningView, (, AutofillPopupVie...","AutofillPopupWarningView,(,AutofillPopupViewNa...",None
...,...,...,...,...,...,...,...,...,...
27722,"[[-0.8682729341000001, -1.2759647071, -1.54760...","[[-0.32964309750000004, -0.0687663168, 0.01822...","[[1, 2, 3], [3, 3, 1], [2, 2, 4], [4, 3, 2]]","[[int, GetLastYear, (), {], [Time, last_year_t...","[[int, FUNC1, (, ), {], [VAR1, VAR2, =, VAR1, ...",[[0]],"[int, GetLastYear, (), {, Time, last_year_time...","int,GetLastYear,(),{,Time,last_year_time,=,Tim...",None
27723,"[[1.5330900732, 0.7245404899, 0.2295006856, -0...","[[0.2159726471, -0.29889355900000003, 0.093106...","[[0, 2, 3], [3, 3, 0], [0, 2, 19], [19, 3, 0],...","[[gpk_compute_crycks, (, sc_card_t, *, card, ,...","[[FUNC1, (, VAR1, *, VAR2, ,, VAR3, *, VAR4, ,...",[[0]],"[gpk_compute_crycks, (, sc_card_t, *, card, ,,...","gpk_compute_crycks,(,sc_card_t,*,card,,,sc_apd...",None
27724,"[[1.3671128926, 0.8307209481000001, -1.3632551...","[[0.10021117140000001, -0.4366644649, 0.025586...","[[0, 2, 5], [5, 3, 0], [0, 2, 7], [7, 3, 0], [...","[[static, int, asepcos_set_sec_attributes, (, ...","[[static, int, FUNC1, (, VAR1, *, VAR2, ,, con...",[[0]],"[static, int, asepcos_set_sec_attributes, (, s...","static,int,asepcos_set_sec_attributes,(,sc_car...",None
27725,"[[0.24357717040000001, 0.1453775644, 0.7804953...","[[0.1413379952, -0.2888208061, 0.0209930599, -...","[[0, 2, 14], [14, 3, 0], [0, 2, 36], [36, 3, 0...","[[after_select, (, fd_set, *, readset, ,, fd_s...","[[FUNC1, (, VAR1, *, VAR2, ,, VAR1, *, VAR3, )...",[[0]],"[after_select, (, fd_set, *, readset, ,, fd_se...","after_select,(,fd_set,*,readset,,,fd_set,*,wri...",None


In [38]:
hit = 0
counter = 0
for index, exp_line_list in gnn_explainer_dict.items():
#     print(index,line_list)
    vul_line = df_test.loc[index].line
    # top k, change k to 1,3,5,10 etc
    exp_line_list = exp_line_list[:10]
#     print(exp_line_list)
    if vul_line != None:
#         print(vul_line)
        found = False
        counter += 1
        origin_token_list = df_test.loc[index].original_tokens
        for exp_line in exp_line_list:
            exp_tokens = origin_token_list[exp_line]
            for exp_token in exp_tokens:
                if len(exp_token) > 1:
                    if exp_token in vul_line:
#                         print(exp_tokens)
#                         print(vul_line)
                        hit += 1
                        found = True
                        break
            if found :
                break
#     break

In [39]:
hit

248

In [40]:
counter

264

In [41]:
hit/counter

0.9393939393939394

In [ ]:
gnn_explainer_dict